In [127]:
import pandas as pd 
import ast
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import tqdm

In [128]:

data_traiter = pd.read_json("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Backend/Datascraping/new_data_json.json")

In [129]:
host = "srv1905.hstgr.io"
user = "u791384722_pharma"
password = "@sCM^n4L"
database = "u791384722_pharma"

In [130]:
request_update = """
UPDATE Medicament SET
    Nom = %s,
    Image_URL = %s,
    Présentation = %s,
    Dosage = %s,
    Distributeur_ou_fabriquant = %s,
    Composition = %s,
    Classe_thérapeutique = %s,
    Statut = %s,
    Code_ATC = %s,
    PPV = %s,
    Prix_hospitalier = %s,
    Tableau = %s,
    Indications = %s,
    Min_Stock = %s,
    Stock_Actuel = %s,
    url_medicament = %s
WHERE Code_EAN_13 = %s;
"""

request_insert ="""INSERT INTO Medicament (
    Code_EAN_13,
    Nom,
    Image_URL,
    Présentation,
    Dosage,
    Distributeur_ou_fabriquant,
    Composition,
    Classe_thérapeutique,
    Statut,
    Code_ATC,
    PPV,
    Prix_hospitalier,
    Tableau,
    Indications,
    Min_Stock,
    Stock_Actuel, 
    url_medicament
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)"""

In [37]:
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )   
for index, values in data_traiter.iterrows():
    values_inser = (
            values['Code_EAN_13'],
            values['Nom'],
            values['Image_URL'],
            values['Présentation'],
            values['Dosage'],
            values['Distributeur_ou_fabriquant'],
            values['Composition'],
            values['Classe_thérapeutique'],
            values['Statut'],
            values['Code_ATC'],
            values['PPV'],
            values['Prix_hospitalier'],
            values['Tableau'],
            values['Indications'],
            values['Min_Stock'],
            values['Stock_Actuel'],
            values['url_medicament'],
        ) 
    values_update = (
            
            values['Nom'],
            values['Image_URL'],
            values['Présentation'],
            values['Dosage'],
            values['Distributeur_ou_fabriquant'],
            values['Composition'],
            values['Classe_thérapeutique'],
            values['Statut'],
            values['Code_ATC'],
            values['PPV'],
            values['Prix_hospitalier'],
            values['Tableau'],
            values['Indications'],
            values['Min_Stock'],
            values['Stock_Actuel'],
            values['url_medicament'],
            values['Code_EAN_13'],
        ) 
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    # 1. Vérifier si le code existe
    cursor.execute("SELECT 1 FROM Medicament WHERE Code_EAN_13 = %s", (values['Code_EAN_13'],))
    exists = cursor.fetchone()

    # 2. Si pas trouvé, insérer
    if not exists:
        print(values['Code_EAN_13'])
        cursor.execute(request_insert, values_inser)
        conn.commit()
    else:
        print(values['Code_EAN_13'], "exist")
        cursor.execute(
            request_update,
            values_update,
        )
        conn.commit()
    

    

6118000121282 exist
6118000480082 exist
611259970274
6111259970281 exist
6118001101368 exist
611259970106
5420024612174 exist
8032578475494 exist
8032578477313
6111259970212 exist
3525720004499 exist
3700221322994 exist
6118001380244 exist
6111261530374 exist


In [139]:
def scrap_medicament(url, code_barre=None, temps_sleep=3):
        options = Options()
        options.add_argument("--headless")  # mode sans interface
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        # Initialisation du navigateur
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(temps_sleep)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")
        data = {}
        h3_tag = soup.find("h3")
        data["Code_EAN_13"] = code_barre if code_barre else None
        data["Nom"] = h3_tag.get_text(strip=True) if h3_tag else None
        img_tag = soup.select_one("div.only-mobile img")
        data["Image URL"] = img_tag["src"] if img_tag else None
        rows = soup.select("table.table-details tr")
        for row in rows:
            field = row.find("td", class_="field")
            value = row.find("td", class_="value")
            if field and value:
                key = field.get_text(strip=True)
                val = value.get_text(strip=True)
                data[key] = val

        def parse_prix(prix_str):
            """Convertit une chaîne de type '36.20 dhs' en float 36.20"""
            if prix_str:
                return float(prix_str.replace("dhs", "").strip())
            return None

        
        data["url"] = url
        return data

In [159]:
url = "https://medicament.ma/medicament/humex-rhume-jn-500-mg-60-mg-500-mg-4-mg-comprime-et-gelule/"

data = scrap_medicament(url)
data['Code_EAN_13'] = 6118000013303
data

{'Code_EAN_13': 6118000013303,
 'Nom': 'HUMEX RHUME, Comprimé et gélule',
 'Image URL': 'https://medicament.ma/image/15791/',
 'Présentation': 'Boîte de 12 comprimés + 4 gélules',
 'Dosage': '500 MG + 60 MG / Ccomprimé + 4 MG / GELULE',
 'Distributeur ou fabriquant': 'MAPHAR',
 'Composition': 'Paracétamol |\xa0Pseudoéphédrine | Chlorphénamine',
 'Statut': 'Commercialisé',
 'Code ATC': 'R01BA52',
 'PPV': '21.90 dhs',
 'Prix hospitalier': '13.70 dhs',
 'Tableau': 'Aucun',
 'Nature du Produit': 'Médicament',
 'url': 'https://medicament.ma/medicament/humex-rhume-jn-500-mg-60-mg-500-mg-4-mg-comprime-et-gelule/'}

In [160]:
Code_EAN_13 = data.get("Code_EAN_13")
Nom = data.get("Nom")
Image_URL = data.get("Image URL")
Présentation = data.get("Présentation")
Dosage = data.get("Dosage")
Distributeur_ou_fabriquant = data.get("Distributeur ou fabriquant")
Composition = data.get("Composition")
Classe_thérapeutique = data.get("Classe thérapeutique")
Statut = data.get("Statut")
Code_ATC = data.get("Code ATC")
PPV = data.get("PPV") if data.get("PPV") else data.get("PPC") if data.get("PPC") else None
Prix_hospitalier = data.get("Prix hospitalier")
Tableau = data.get("Tableau")
Indications = data.get("Indication(s)")
Min_Stock = 0
Stock_Actuel = 0
url_medicament = data.get("url")


values_update = (
                Nom,
                Image_URL,
                Présentation,
                Dosage,
                Distributeur_ou_fabriquant,
                Composition,
                Classe_thérapeutique,
                Statut,
                Code_ATC,
                PPV,
                Prix_hospitalier,
                Tableau,
                Indications,
                Min_Stock,
                Stock_Actuel,
                url_medicament,
                Code_EAN_13,)
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                ) 
cursor = conn.cursor(pymysql.cursors.DictCursor)
cursor.execute(
            request_update,
            values_update,
        )
conn.commit()

In [ ]:
[6118001185726,
 6118001185733,
 6118000280767,
 6111273630017,
 6118001185719,
 6111269050034,
 6118001185719,
 6118001185733]

In [123]:
# add new data
import os
import json
import numpy as np
medicaments = pd.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/Medicament (2).csv")
def lire_tous_les_json(dossier):
    all_data = []
    for nom_fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, nom_fichier)
        if nom_fichier.endswith('.json') and os.path.isfile(chemin_fichier):
            with open(chemin_fichier, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    if isinstance(data, list):
                        all_data.extend(data)
                except json.JSONDecodeError:
                    print(f"[ERREUR] Le fichier {nom_fichier} n'est pas un JSON valide.")
    return all_data

# Exemple d'utilisation
dossier_path = "/Users/ahmedzaiou/Documents/ProjetsApps/MiseajourMedicament/data_from_eservice"
resultats = lire_tous_les_json(dossier_path)
print(f"{len(resultats)} objets JSON lus au total.")
new_scraped = pd.DataFrame(resultats)
diff = list(set(new_scraped['code_ean_13'].values) - set(medicaments['Code_EAN_13'].values))
new_data_to_add = new_scraped[new_scraped['code_ean_13'].isin(diff)]
print(new_data_to_add.shape)

5673 objets JSON lus au total.
(1714, 9)


In [126]:
while True:
    code = input("Enter the code EAN 13 to check if it exists in the new data to add (or type 'exit' to quit): ")
    if code.lower() == 'exit':
        break
    if code in new_data_to_add['code_ean_13'].values:
        print(f"The code {code} exists in the new data to add.")
    else:
        print(f"The code {code} does not exist in the new data to add.")

The code 3760049895285 does not exist in the new data to add.
The code 4028694000829 does not exist in the new data to add.
The code 6111261530275 does not exist in the new data to add.
The code 6111269050119 does not exist in the new data to add.
The code 6111269050157 does not exist in the new data to add.
The code 6118001380268 does not exist in the new data to add.
The code 6118001380275 does not exist in the new data to add.
The code 6111260760031 does not exist in the new data to add.
The code 6487595253563 does not exist in the new data to add.
The code 648595253525 does not exist in the new data to add.
The code 6111261530022 does not exist in the new data to add.


to use for the test : 6111269050119